In [1]:
%matplotlib inline

# include modules to the path
import sys, os
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parent_dir)
sys.path.append(os.path.join(parent_dir, 'session'))

import os
import numpy as np
import h5py, json

from session.utils import get_sessions_list, get_sampling_rate, cleaned_epochs

In [2]:
source = '/home/sobolev/nevermind/Andrey/data'
animal = '009266'

## Manually patching JSON files

In [23]:
for session_name in os.listdir(os.path.join(source, animal)):
    # file with session config
    f_name = os.path.join(source, animal, session_name, session_name + '.json')

    with open(f_name, 'r') as f:
        cfg = json.load(f)

    # update required fields
    cfg['experiment']['timepoints'] = []
    cfg['experiment']['light_events'] = []

    with open(f_name, 'w', encoding='utf-8') as f:
        json.dump(cfg, f, ensure_ascii=False, indent=4)

## Clean up

In [3]:
to_clean = ['fil', 'lfp', 'clu', 'fet', 'klg', 'res', 'spk', 'nrs', 'xml']

count = 0
sessions = [x for x in os.listdir(os.path.join(source, animal)) if os.path.isdir(os.path.join(source, animal, x))]
for session_name in sessions:
    for file_name in os.listdir(os.path.join(source, animal, session_name)):
        for key in to_clean:
            if file_name.find('.' + key) > 0:
                os.remove(os.path.join(source, animal, session_name, file_name))
                count += 1
                
print('Removed %s files' % count)

Removed 84 files


In [30]:
def is_wrong(file_name):
    keys = ['clu', 'fet', 'klg', 'res', 'spk', 'nrs', 'xml']
    for key in keys:
        if file_name.find('.' + key) > 0:
            return True
    return False

from functools import reduce
sessions = [x for x in os.listdir(os.path.join(source, animal)) if os.path.isdir(os.path.join(source, animal, x))]
files_all = reduce(lambda x, y: x + y, [os.listdir(os.path.join(source, animal, session)) for session in sessions])
'foo'

'foo'

## Put correct .XML

In [5]:
import shutil

sessions = [x for x in os.listdir(os.path.join(source, animal)) if os.path.isdir(os.path.join(source, animal, x))]

for session_name in sessions:
    for file_name in os.listdir(os.path.join(source, animal, session_name)):
        if file_name.find('.dat') > 0:
            src = os.path.join(source, animal, 'default.xml')
            dst = os.path.join(source, animal, session_name, session_name + '.xml')
            shutil.copyfile(src, dst)

In [6]:
sessions

['009266_hippoSIT_2023-02-28_19-53-56',
 '009266_hippoSIT_2023-03-01_17-53-11',
 '009266_hippoSIT_2023-03-06_15-10-36',
 '009266_hippoSIT_2023-03-06_20-43-19',
 '009266_hippoSIT_2023-03-08_17-06-45',
 '009266_hippoSIT_2023-03-09_09-37-07',
 '009266_hippoSIT_2023-03-09_19-12-22']

## Sandbox

In [6]:
session_name = os.listdir(os.path.join(source, animal))[-1]

cleaned_epochs(os.path.join(source, animal, session_name))

array([[   0., 2400.]])

In [7]:
a1 = '009265_hippoSIT_2023-03-05_18-31-32.xml'
a1.find('xml')

36

## Copy H5 from selected sessions to a separate folder

In [5]:
import shutil

source = '/home/sobolev/nevermind/Andrey/data'
dest = '/home/sobolev/nevermind/Andrey/sandbox/export'

selected_sessions = [
'009266_hippoSIT_2023-04-17_17-04-17',  # ch17, 20 + 55 correction, 5067 events. Showcase for N2 / N3 mod in target
'009266_hippoSIT_2023-04-18_10-10-37',  # ch17, 10 + 55 correction, 5682 events
'009266_hippoSIT_2023-04-18_17-03-10',  # ch17, 6 + 55 correction, 5494 events: FIXME very weird 1-2nd in target, find out
'009266_hippoSIT_2023-04-19_10-33-51',  # ch17, 4 + 55 correction, 6424 events: very weird 1-2nd in target, find out
'009266_hippoSIT_2023-04-20_08-57-39',  # ch1, 1 + 55 correction, 6424 events. Showcase for N2 / N3 mod in target
'009266_hippoSIT_2023-04-24_16-56-55',  # ch17, 5 + 55* correction, 6165 events, frequency
'009266_hippoSIT_2023-04-26_08-20-17',  # ch17, 12 + 55* correction, 6095 events, duration - showcase for N2 
'009266_hippoSIT_2023-05-02_12-22-14',  # ch20, 10 + 55 correction, 5976 events, FIXME very weird 1-2nd in target, find out
'009266_hippoSIT_2023-05-04_09-11-06',  # ch17, 5 + 55* correction, 4487 events, coma session with baseline AEPs
#'009266_hippoSIT_2023-05-04_19-47-15',  # ch20, 2 + 55 correction, 5678 events, duration
]

In [4]:
for session in selected_sessions:
    animal      = session.split('_')[0]

    # create folder
    session_src = os.path.join(source, animal, session)
    session_dst = os.path.join(dest, animal, session)
    if not os.path.exists(session_dst):
        os.makedirs(session_dst)
        
    # copy files
    for f_name in ['AEPs.h5', session + '.h5']:
        src = os.path.join(session_src, f_name)
        dst = os.path.join(session_dst, f_name)
        shutil.copyfile(src, dst)